In [1]:
import torch

#model = torch.load("../runs/train/yolov7/weights/best.pt").to(device)
device = torch.device("cuda")
model = torch.load("../runs/train/yolov7-prune0.5/weights/best.pt")['model'].to(device)
torch.save(model.state_dict(), "../runs/train/yolov7-prune0.5/weights/best_state_dict.pt")
#print(torch.__version__)

FileNotFoundError: [Errno 2] No such file or directory: '../runs/train/yolov7-prune0.5/weights/best.pt'

In [ ]:
# 训练模型的项目的根目录（可以是与当前文件的相对路径）
import sys
sys.path.insert(0, '../')

import torch
import torch.nn.functional as F
from torch.optim import SGD
from models.yolo import Model

from nni_assets.compression.mnist_model import TorchModel, trainer, evaluator, device

# define the model
device = torch.device("cuda")
model = Model("../cfg/training/yolov7_class12_prunning.yaml").to(device)  # create
model.load_state_dict(torch.load("./best_state_dict.pt"), strict=False)

# show the model structure, note that pruner will wrap the model layer.
#print(model)

In [ ]:
config_list = [{
    'sparsity_per_layer': 0.5,
    'op_types': ['Linear', 'Conv2d']
    #'op_names': ['model.0.conv']
}]

In [ ]:
from nni.compression.pytorch.pruning import L1NormPruner
pruner = L1NormPruner(model, config_list)

# show the wrapped model structure, `PrunerModuleWrapper` have wrapped the layers that configured in the config_list.
#print(model)

In [ ]:
# compress the model and generate the masks
_, masks = pruner.compress()
# show the masks sparsity
for name, mask in masks.items():
    #print(name, ' shape:',  mask['weight'].shape)
    print(name, ' sparsity : ', '{:.2}'.format(mask['weight'].sum(dtype=torch.float32) / mask['weight'].numel()))

In [ ]:
# need to unwrap the model, if the model is wrapped before speedup
pruner.show_pruned_weights()
pruner._unwrap_model()
#input = torch.rand(1, 3, 640, 640).to(device)
#traced_scrpit_modele = torch.jit.trace(model, input)
#print(traced_scrpit_modele.code)

In [ ]:
from nni.compression.pytorch import ModelSpeedup
ModelSpeedup(model, dummy_input=torch.rand([1, 3, 640, 640]).to(device), masks_file=masks).speedup_model()